In [21]:
import pandas as pd
import kagglehub
from pandasql import sqldf


# Last ned datasettene
pathGermany = kagglehub.dataset_download("l3llff/wind-power")

# Les inn CSV-filene som DataFrames
df_germany = pd.read_csv(f"{pathGermany}/data.csv")



Importerer det jeg trenger til oppgave 3, del 1

In [28]:
#Undersøker om det finnes manglede dager

df_germany['dt'] = pd.to_datetime(df_germany['dt'], errors='coerce')

df_germany['d'] = df_germany['dt'].dt.date

alle_dager = pd.date_range(start=df_germany['d'].min(), end=df_germany['d'].max(), freq='d')
manglende_dager = alle_dager.difference(df_germany['d'])


if not manglende_dager.empty: #Hvis den ikke er tom så skriv:
    print(f"Følgende dager mangler: {manglende_dager}")
else: #Om den er tom så skriv:
    print("Ingen manglende dager")


Ingen manglende dager


Ser at det er ingen hele dager som mangler

In [ ]:
#Her finner koden hvilken 15 min intervaller som mangler 

df_germany['datetime'] = pd.to_datetime(df_germany['dt'], errors='coerce')

# Her brukes list comprehension for å lage intervaller på 15 minutter
alle_intervaller_df = pd.DataFrame({
    'datetime': pd.date_range(start=df_germany['datetime'].min(),
                              end=df_germany['datetime'].max(),
                              freq='15T')
})

# Finner mangler med sql
query = """
    SELECT a.datetime
    FROM alle_intervaller_df a
    LEFT JOIN df_germany b
    ON a.datetime = b.datetime
    WHERE b.datetime IS NULL
"""
manglende_intervaller_df = sqldf(query, locals())

# List comprehension for å legge dataene i en strukturert DataFrame
manglende_intervaller = [
    {'Manglende intervaller': tidspunkt} 
    for tidspunkt in manglende_intervaller_df['datetime']
]

manglende_intervaller_df = pd.DataFrame(manglende_intervaller)

# Sjekker om det finnes manglende intervaller
if not manglende_intervaller_df.empty:
    print("\nFølgende 15-minutters intervaller mangler:\n")
    print(manglende_intervaller_df.to_string(index=False))
    print(f"\nAntall manglende 15-minutters intervaller: {len(manglende_intervaller_df)}")
    
    # Lagre resultatene i en CSV-fil
    manglende_intervaller_df.to_csv("manglende_intervaller.csv", index=False)
    print("\nManglende intervaller er også lagret i 'manglende_intervaller.csv'.")
else:
    print("Ingen manglende 15-minutters intervaller")


/var/folders/8f/nwn4bqq56715k1k5jg6vggg00000gn/T/ipykernel_62373/4250959714.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'datetime': pd.date_range(start=df_germany['datetime'].min(),



Følgende 15-minutters intervaller mangler:

     Manglende intervaller
2011-03-27 02:00:00.000000
2011-03-27 02:15:00.000000
2011-03-27 02:30:00.000000
2011-03-27 02:45:00.000000
2012-03-25 02:00:00.000000
2012-03-25 02:15:00.000000
2012-03-25 02:30:00.000000
2012-03-25 02:45:00.000000
2013-03-31 02:00:00.000000
2013-03-31 02:15:00.000000
2013-03-31 02:30:00.000000
2013-03-31 02:45:00.000000
2014-03-30 02:00:00.000000
2014-03-30 02:15:00.000000
2014-03-30 02:30:00.000000
2014-03-30 02:45:00.000000
2014-04-07 03:30:00.000000
2014-06-20 14:45:00.000000
2014-06-20 15:30:00.000000
2014-06-25 11:30:00.000000
2014-06-25 13:45:00.000000
2014-06-25 14:15:00.000000
2015-03-29 02:00:00.000000
2015-03-29 02:15:00.000000
2015-03-29 02:30:00.000000
2015-03-29 02:45:00.000000
2016-03-27 02:00:00.000000
2016-03-27 02:15:00.000000
2016-03-27 02:30:00.000000
2016-03-27 02:45:00.000000
2017-03-26 02:00:00.000000
2017-03-26 02:15:00.000000
2017-03-26 02:30:00.000000
2017-03-26 02:45:00.000000
2018-03-25

Nå ser vi at det er 52 15-minutters intervaller som mangler. 



In [ ]:
#Her finner koden om det er noen dublikater

# Sorter dataene etter tid for å få riktig rekkefølge
df_germany = df_germany.sort_values('dt').reset_index(drop=True)

df_germany['datetime'] = pd.to_datetime(df_germany['dt'], errors='coerce')

duplikater = df_germany[df_germany.duplicated('datetime', keep=False)]

# Sjekker om det finnes noen duplikater
if not duplikater.empty:

    print("\nFølgende datoer og tidspunkter forekommer flere ganger:\n")
    print(duplikater.to_string(index=False))
    
    # Skriver ut antall ganger hver duplikat blir funnet
    antall_duplikater = duplikater['datetime'].value_counts().reset_index()
    antall_duplikater.columns = ['datetime', 'Antall forekomster']

    print("\nAntall ganger hvert tidspunkt forekommer:\n")
    print(antall_duplikater.to_string(index=False))
    
    # Lagrer resultatene i en CSV-fil til bruk senere
    antall_duplikater.to_csv("dupliserte_tidspunkter.csv", index=False)
    print("\nDuplikatene er også lagret i 'dupliserte_tidspunkter.csv'.")
else:
    print("Ingen dupliserte tidspunkter funnet.")



Følgende datoer og tidspunkter forekommer flere ganger:

                 dt       MW          d            datetime
2011-10-30 02:00:00  1034.00 2011-10-30 2011-10-30 02:00:00
2011-10-30 02:00:00  1159.00 2011-10-30 2011-10-30 02:00:00
2011-10-30 02:15:00  1012.00 2011-10-30 2011-10-30 02:15:00
2011-10-30 02:15:00  1110.00 2011-10-30 2011-10-30 02:15:00
2011-10-30 02:30:00   974.00 2011-10-30 2011-10-30 02:30:00
2011-10-30 02:30:00  1078.00 2011-10-30 2011-10-30 02:30:00
2011-10-30 02:45:00  1059.00 2011-10-30 2011-10-30 02:45:00
2011-10-30 02:45:00  1025.00 2011-10-30 2011-10-30 02:45:00
2012-10-28 02:00:00  1582.00 2012-10-28 2012-10-28 02:00:00
2012-10-28 02:00:00  1542.00 2012-10-28 2012-10-28 02:00:00
2012-10-28 02:15:00  1631.00 2012-10-28 2012-10-28 02:15:00
2012-10-28 02:15:00  1437.00 2012-10-28 2012-10-28 02:15:00
2012-10-28 02:30:00  1523.00 2012-10-28 2012-10-28 02:30:00
2012-10-28 02:30:00  1631.00 2012-10-28 2012-10-28 02:30:00
2012-10-28 02:45:00  1373.00 2012-10-28 20

In [ ]:
#Her lages det en ny csv fil som inneholder det komplette datasettet. Med informasjonen som mangler, samt uten dublikerte tidspunkter. 

#Tidsrekke med 15-minutters intervaller
alle_intervaller = pd.date_range(start=df_germany['datetime'].min(),
                                 end=df_germany['datetime'].max(),
                                 freq='15T')

#finner manglende intervaller
manglende_intervaller = alle_intervaller.difference(df_germany['datetime'])

#Lager en DataFrame med manglende intervaller og sett inn med gjennomsnittet av verdien over og under
manglende_data = pd.DataFrame([
    {
        'datetime': tidspunkt,
        'MW': (df_germany.loc[df_germany['datetime'] < tidspunkt, 'MW'].iloc[-1] +
               df_germany.loc[df_germany['datetime'] > tidspunkt, 'MW'].iloc[0]) / 2
    }
    for tidspunkt in manglende_intervaller
])

#Kombinerer
df_komplett = pd.concat([df_germany, manglende_data]).sort_values('datetime').reset_index(drop=True)

#Fjern duplikater, men behold gjennomsnittet av MW-verdiene for dublikater
df_komplett = df_komplett.groupby('datetime', as_index=False).agg({'MW': 'mean'})

#Lagre resultatet til CSV
df_komplett.to_csv("komplett_data_med_utfylte_mengder.csv", index=False)

print(" Data med utfylte verdier og fjernede duplikater er lagret som 'komplett_data_med_utfylte_mengder.csv'.")


/var/folders/8f/nwn4bqq56715k1k5jg6vggg00000gn/T/ipykernel_62373/1516637141.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  alle_intervaller = pd.date_range(start=df_germany['datetime'].min(),


 Data med utfylte verdier og fjernede duplikater er lagret som 'komplett_data_med_utfylte_mengder.csv'.


In [44]:
#Importerer den nye csv filen for å kunne bruke den videre:

df_komplett = pd.read_csv("komplett_data_med_utfylte_mengder.csv")
#Konverter 'dt'-kolonnen tilbake til datetime-format
df_komplett['datetime'] = pd.to_datetime(df_komplett['datetime'])

print(df_komplett.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385568 entries, 0 to 385567
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  385568 non-null  datetime64[ns]
 1   MW        385568 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.9 MB
None
